In [1]:
import pandas as pd
import numpy as np
import re
import random

In [2]:
df = pd.read_excel('../recordFPT - recode/base/4process_25Nov.xlsx')

# preprocess

In [3]:
df = df.dropna(subset=['KL_flow'])
df['KL_flow'].isna().sum()
df.drop(columns=['DV_Số điện thoại', 'DV_Số hợp đồng', 'DV_Dịch vụ',
       'TT_Vùng', 'TT_Giới tính', 'TT_Nhóm tuổi', 'TT_Mức độ am hiểu',
       'TT_Mục đích sử dụng', 'TT_Thời gian sử dụng', 'TT_Note','KL_Issue', 'KL_Success', 'KL_Warning',
       'KL_Consideration', 'KL_Đánh giá NV + đề xuất'],
        inplace=True)

In [4]:
df

,STT,KL_Nội dung hỗ trợ,KL_flow
0,1,Thanh lý hợp đồng internet,n12=s1>ds1\s2>ds3\s3>dx5\nn34=s1>hd5
1,2,Thanh lý hợp đồng internet,n00=s1>ds1+dx4\nn10=s1>ds3\s2>ds1\s3>ll3+dx5\n...
2,3,Tạm ngưng hợp đồng internet,n00=s1>gt1+ds1\nn22=s1>ll3+dx4\s2>dx5\nn34=s1>...
3,4,Thanh lý hợp đồng internet,n16=s1>ds4+dx5\s2>ll4+dx5\nn34=s1>hd1+hd5
4,5,Thanh lý hợp đồng internet,n00=s1>ds1\s2>ds3+dx4\nn12=s1>ds4\nn11=s1>dx7\...
...,...,...,...
679,680,Thanh lý hợp đồng truyền hình trong combo,n20=s1>ll2\nn53=s1>ll2+ll5+ll18\s2>hd4[giga bỏ...
680,681,Tư vấn về cước,n37#=s1'>hd5+gt10
681,682,Thanh lý hợp đồng internet,n00=s1'>ds1\nn19=s1>dx4\s2>ll5+dx5\s3>ds2\s4>d...
682,683,Thanh lý hợp đồng truyền hình trong combo,n00=s1'>ds1+ll5\nn21[truyền hình]=s1'>ll2+ds2\...


In [5]:
def split_and_index_kl_flow(df):
    
    df['KL_flow'] = df['KL_flow'].astype(str)

    # Create a new dataframe to store the processed data
    processed_data = []
    
    # Iterate through each row in the original dataframe
    for idx, row in df.iterrows():
        stt = row['STT']
        kl_flow = row['KL_flow']
        
        # Split the KL_flow into multiple lines based on '\n'
        flow_lines = kl_flow.split('\n')
        
        # Iterate through each flow line and create sub_ids
        for sub_id, flow in enumerate(flow_lines, start=1):
            new_row = row.copy()  # Copy the row to keep other columns the same
            new_row['sub_id'] = f"{stt}.{sub_id}"  # Create sub_id by combining STT and sub_id
            new_row['KL_flow'] = flow  # Assign the current flow line to KL_flow
            processed_data.append(new_row)
    
    # Convert the processed data into a new DataFrame
    processed_df = pd.DataFrame(processed_data)
    
    # Reorder columns: Insert sub_id right after STT
    cols = list(processed_df.columns)
    cols.insert(1, cols.pop(cols.index('sub_id')))  # Move sub_id to be the second column
    processed_df = processed_df[cols]
    
    # Reset the index
    processed_df.reset_index(drop=True, inplace=True)
    
    return processed_df

In [6]:
df = split_and_index_kl_flow(df)

In [7]:
df.columns

Index(['STT', 'sub_id', 'KL_Nội dung hỗ trợ', 'KL_flow'], dtype='object')

In [8]:
df

,STT,sub_id,KL_Nội dung hỗ trợ,KL_flow
0,1,1.1,Thanh lý hợp đồng internet,n12=s1>ds1\s2>ds3\s3>dx5
1,1,1.2,Thanh lý hợp đồng internet,n34=s1>hd5
2,2,2.1,Thanh lý hợp đồng internet,n00=s1>ds1+dx4
3,2,2.2,Thanh lý hợp đồng internet,n10=s1>ds3\s2>ds1\s3>ll3+dx5
4,2,2.3,Thanh lý hợp đồng internet,n34=s1>hd1+hd5
...,...,...,...,...
1234,683,683.2,Thanh lý hợp đồng truyền hình trong combo,n21[truyền hình]=s1'>ll2+ds2
1235,683,683.3,Thanh lý hợp đồng truyền hình trong combo,n20[internet]=s1'>gt3+dx9[combo giga cộng tru...
1236,684,684.1,Thanh lý hợp đồng truyền hình trong combo,n21#[truyền hình]=s1!>ll1+ll2+ll16+gt1
1237,684,684.2,Thanh lý hợp đồng truyền hình trong combo,n50=s1>ds1\s2>ll3+ll5>s3>dx10[6+1 hoặc 12+2]


In [9]:
def extract_issue(df):
    # Create a list to store split_flow values
    split_flow_list = []
    
    for index, row in df.iterrows():
        flow_value = row['KL_flow']
        
        # Check if the value is a string and not NaN
        if isinstance(flow_value, str):
            split_flow = flow_value.split("=", 1)[0]
            
            if "#" in split_flow:
                split_flow = split_flow.replace("#", "")

            if "[" in split_flow:
                split_flow = split_flow.split("[", 1)[0]
                
            split_flow_list.append(split_flow)  # Append to the list
        else:
            split_flow_list.append(None)  # Append None for invalid values
            print(f"Row {index}: Invalid Flow value (not a string): {flow_value}")

    # Add the new column to the DataFrame
    df['Reason'] = split_flow_list

    return df

In [10]:
df = extract_issue(df)

In [11]:
df

,STT,sub_id,KL_Nội dung hỗ trợ,KL_flow,Reason
0,1,1.1,Thanh lý hợp đồng internet,n12=s1>ds1\s2>ds3\s3>dx5,n12
1,1,1.2,Thanh lý hợp đồng internet,n34=s1>hd5,n34
2,2,2.1,Thanh lý hợp đồng internet,n00=s1>ds1+dx4,n00
3,2,2.2,Thanh lý hợp đồng internet,n10=s1>ds3\s2>ds1\s3>ll3+dx5,n10
4,2,2.3,Thanh lý hợp đồng internet,n34=s1>hd1+hd5,n34
...,...,...,...,...,...
1234,683,683.2,Thanh lý hợp đồng truyền hình trong combo,n21[truyền hình]=s1'>ll2+ds2,n21
1235,683,683.3,Thanh lý hợp đồng truyền hình trong combo,n20[internet]=s1'>gt3+dx9[combo giga cộng tru...,n20
1236,684,684.1,Thanh lý hợp đồng truyền hình trong combo,n21#[truyền hình]=s1!>ll1+ll2+ll16+gt1,n21
1237,684,684.2,Thanh lý hợp đồng truyền hình trong combo,n50=s1>ds1\s2>ll3+ll5>s3>dx10[6+1 hoặc 12+2],n50


In [12]:
df

,STT,sub_id,KL_Nội dung hỗ trợ,KL_flow,Reason
0,1,1.1,Thanh lý hợp đồng internet,n12=s1>ds1\s2>ds3\s3>dx5,n12
1,1,1.2,Thanh lý hợp đồng internet,n34=s1>hd5,n34
2,2,2.1,Thanh lý hợp đồng internet,n00=s1>ds1+dx4,n00
3,2,2.2,Thanh lý hợp đồng internet,n10=s1>ds3\s2>ds1\s3>ll3+dx5,n10
4,2,2.3,Thanh lý hợp đồng internet,n34=s1>hd1+hd5,n34
...,...,...,...,...,...
1234,683,683.2,Thanh lý hợp đồng truyền hình trong combo,n21[truyền hình]=s1'>ll2+ds2,n21
1235,683,683.3,Thanh lý hợp đồng truyền hình trong combo,n20[internet]=s1'>gt3+dx9[combo giga cộng tru...,n20
1236,684,684.1,Thanh lý hợp đồng truyền hình trong combo,n21#[truyền hình]=s1!>ll1+ll2+ll16+gt1,n21
1237,684,684.2,Thanh lý hợp đồng truyền hình trong combo,n50=s1>ds1\s2>ll3+ll5>s3>dx10[6+1 hoặc 12+2],n50


In [13]:
def extract_solution(df):
    processed_data = []
    
    for index, row in df.iterrows():
        # print('working on: ' + str(index))
        
        case_id = row['STT']
        flow_id = row['sub_id']
        flow_value = row['KL_flow']

        # Split the 'Flow' value based on '=' and then '\'
        solution_lines = flow_value.split('=')[1] if '=' in flow_value else flow_value
        solution_lines = solution_lines.split('\\')

        # print(solution_lines)

        # Iterate through each solution line
        for solution_id, item in enumerate(solution_lines, start=1):
            solution = item.split('>')[0]
            protocols = item.split('>')[1] if '>' in item else ''

            # print(protocols)

            # Check for symbols (^, !, ') and split accordingly
            if '^' in solution:
                just_a_solution = solution.split('^')[0]
                solution_result = 'Cân nhắc'
            elif '!' in solution:
                just_a_solution = solution.split('!')[0]
                solution_result = 'Lưu ý'
            elif "'" in solution:
                just_a_solution = solution.split("'")[0]
                solution_result = "Thành công"
            else:
                just_a_solution = solution  # No symbol found, keep the solution as is
                solution_result = "Không thành công"  # No symbol means no result     

            # Add the main solution row
            new_row = row.copy()  # Copy the row to keep other columns the same
            new_row['solution_id'] = f"{flow_id}.{solution_id}" 
            new_row['Solution'] = just_a_solution
            new_row['Solution Result'] = solution_result
            new_row['Combination'] = protocols

            # Append the main solution row to processed_data
            processed_data.append(new_row)
            
          
    # Convert the processed data into a new DataFrame
    processed_df = pd.DataFrame(processed_data)
    
    # Reorder columns: Insert sub_id right after STT
    cols = list(processed_df.columns)
    processed_df = processed_df[cols]
    
    # Reset the index
    processed_df.reset_index(drop=True, inplace=True)
                    
    return processed_df

In [14]:
df = extract_solution(df)

In [15]:
df

,STT,sub_id,KL_Nội dung hỗ trợ,KL_flow,Reason,solution_id,Solution,Solution Result,Combination
0,1,1.1,Thanh lý hợp đồng internet,n12=s1>ds1\s2>ds3\s3>dx5,n12,1.1.1,s1,Không thành công,ds1
1,1,1.1,Thanh lý hợp đồng internet,n12=s1>ds1\s2>ds3\s3>dx5,n12,1.1.2,s2,Không thành công,ds3
2,1,1.1,Thanh lý hợp đồng internet,n12=s1>ds1\s2>ds3\s3>dx5,n12,1.1.3,s3,Không thành công,dx5
3,1,1.2,Thanh lý hợp đồng internet,n34=s1>hd5,n34,1.2.1,s1,Không thành công,hd5
4,2,2.1,Thanh lý hợp đồng internet,n00=s1>ds1+dx4,n00,2.1.1,s1,Không thành công,ds1+dx4
...,...,...,...,...,...,...,...,...,...
2030,683,683.3,Thanh lý hợp đồng truyền hình trong combo,n20[internet]=s1'>gt3+dx9[combo giga cộng tru...,n20,683.3.1,s1,Thành công,gt3+dx9[combo giga cộng truyền hình]+hd4[comb...
2031,684,684.1,Thanh lý hợp đồng truyền hình trong combo,n21#[truyền hình]=s1!>ll1+ll2+ll16+gt1,n21,684.1.1,s1,Lưu ý,ll1+ll2+ll16+gt1
2032,684,684.2,Thanh lý hợp đồng truyền hình trong combo,n50=s1>ds1\s2>ll3+ll5>s3>dx10[6+1 hoặc 12+2],n50,684.2.1,s1,Không thành công,ds1
2033,684,684.2,Thanh lý hợp đồng truyền hình trong combo,n50=s1>ds1\s2>ll3+ll5>s3>dx10[6+1 hoặc 12+2],n50,684.2.2,s2,Không thành công,ll3+ll5


In [16]:
df.columns

Index(['STT', 'sub_id', 'KL_Nội dung hỗ trợ', 'KL_flow', 'Reason',
       'solution_id', 'Solution', 'Solution Result', 'Combination'],
      dtype='object')

In [17]:
df = df.rename(columns={'STT': 'case_id', 'sub_id': 'issue_id', 'KL_Nội dung hỗ trợ': 'Nội dung hỗ trợ'})

In [18]:
df

,case_id,issue_id,Nội dung hỗ trợ,KL_flow,Reason,solution_id,Solution,Solution Result,Combination
0,1,1.1,Thanh lý hợp đồng internet,n12=s1>ds1\s2>ds3\s3>dx5,n12,1.1.1,s1,Không thành công,ds1
1,1,1.1,Thanh lý hợp đồng internet,n12=s1>ds1\s2>ds3\s3>dx5,n12,1.1.2,s2,Không thành công,ds3
2,1,1.1,Thanh lý hợp đồng internet,n12=s1>ds1\s2>ds3\s3>dx5,n12,1.1.3,s3,Không thành công,dx5
3,1,1.2,Thanh lý hợp đồng internet,n34=s1>hd5,n34,1.2.1,s1,Không thành công,hd5
4,2,2.1,Thanh lý hợp đồng internet,n00=s1>ds1+dx4,n00,2.1.1,s1,Không thành công,ds1+dx4
...,...,...,...,...,...,...,...,...,...
2030,683,683.3,Thanh lý hợp đồng truyền hình trong combo,n20[internet]=s1'>gt3+dx9[combo giga cộng tru...,n20,683.3.1,s1,Thành công,gt3+dx9[combo giga cộng truyền hình]+hd4[comb...
2031,684,684.1,Thanh lý hợp đồng truyền hình trong combo,n21#[truyền hình]=s1!>ll1+ll2+ll16+gt1,n21,684.1.1,s1,Lưu ý,ll1+ll2+ll16+gt1
2032,684,684.2,Thanh lý hợp đồng truyền hình trong combo,n50=s1>ds1\s2>ll3+ll5>s3>dx10[6+1 hoặc 12+2],n50,684.2.1,s1,Không thành công,ds1
2033,684,684.2,Thanh lý hợp đồng truyền hình trong combo,n50=s1>ds1\s2>ll3+ll5>s3>dx10[6+1 hoặc 12+2],n50,684.2.2,s2,Không thành công,ll3+ll5


In [19]:
nn = pd.read_excel('../recordFPT - recode/base/label.xlsx', sheet_name='nguyên nhân')

issue_map = dict(zip(nn['Tên nguyên nhân'], nn['Mã nhóm nguyên nhân']))
df['Reason Group'] = df['Reason'].map(issue_map)

In [20]:
df = df[['case_id', 'Nội dung hỗ trợ', 'Reason', 'Reason Group', 'Solution', 'Combination', 'Solution Result']]

In [21]:
df

,case_id,Nội dung hỗ trợ,Reason,Reason Group,Solution,Combination,Solution Result
0,1,Thanh lý hợp đồng internet,n12,G1,s1,ds1,Không thành công
1,1,Thanh lý hợp đồng internet,n12,G1,s2,ds3,Không thành công
2,1,Thanh lý hợp đồng internet,n12,G1,s3,dx5,Không thành công
3,1,Thanh lý hợp đồng internet,n34,G3,s1,hd5,Không thành công
4,2,Thanh lý hợp đồng internet,n00,G0,s1,ds1+dx4,Không thành công
...,...,...,...,...,...,...,...
2030,683,Thanh lý hợp đồng truyền hình trong combo,n20,G2,s1,gt3+dx9[combo giga cộng truyền hình]+hd4[comb...,Thành công
2031,684,Thanh lý hợp đồng truyền hình trong combo,n21,G2,s1,ll1+ll2+ll16+gt1,Lưu ý
2032,684,Thanh lý hợp đồng truyền hình trong combo,n50,G5,s1,ds1,Không thành công
2033,684,Thanh lý hợp đồng truyền hình trong combo,n50,G5,s2,ll3+ll5,Không thành công


In [22]:
df['Combination'] = df['Combination'].str.replace(r'\[.*?\]', '', regex=True)

C:\Users\anhthy\AppData\Local\Temp\ipykernel_2184\2919777391.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Combination'] = df['Combination'].str.replace(r'\[.*?\]', '', regex=True)


In [23]:
# df.to_excel("store/base_combination.xlsx")

# processing reason

# count how many protocol in solution

In [24]:
def how_many_protocol(df):
    counts = []
    for index, row in df.iterrows():
        combine = row['Combination']
        
        # Check if 'Combine' is None or not a string, and handle it appropriately
        if not isinstance(combine, str):
            print(f"Skipping row at index {index} due to None or non-string Combine value.")
            print(row['Flow'])
            counts.append(0)  # Assign 0 if Combine is None or not a string
            continue
        
        # Count occurrences of '+' and ':' and add 1
        total_count = sum(1 for char in combine if char in ['+', ':']) + 1
        counts.append(total_count)

    # Add the new column to the DataFrame
    df['How many protocol in solution'] = counts
    return df

In [25]:
df = how_many_protocol(df)

C:\Users\anhthy\AppData\Local\Temp\ipykernel_2184\72771589.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['How many protocol in solution'] = counts


In [26]:
df.head(5)

,case_id,Nội dung hỗ trợ,Reason,Reason Group,Solution,Combination,Solution Result,How many protocol in solution
0,1,Thanh lý hợp đồng internet,n12,G1,s1,ds1,Không thành công,1
1,1,Thanh lý hợp đồng internet,n12,G1,s2,ds3,Không thành công,1
2,1,Thanh lý hợp đồng internet,n12,G1,s3,dx5,Không thành công,1
3,1,Thanh lý hợp đồng internet,n34,G3,s1,hd5,Không thành công,1
4,2,Thanh lý hợp đồng internet,n00,G0,s1,ds1+dx4,Không thành công,2


# Normalize by alphabet. For example: ll2+dx3+comor3 = comor3+dx3+ll2

In [27]:
def normalize(df):
    # Step 1: Normalize each combination by sorting parts around '+'
    normalized_combinations = []
    for combination in df['Combination']:
        parts = sorted(combination.split('+'))  # Sort parts by '+'
        normalized_combination = '+'.join(parts)  # Join parts back together
        normalized_combinations.append(normalized_combination)
    
    # Step 2: Save the normalized combinations back to the DataFrame
    df['Combination'] = normalized_combinations  # Add new column to store normalized values
    
    return df

In [28]:
df = normalize(df)

C:\Users\anhthy\AppData\Local\Temp\ipykernel_2184\2538986675.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Combination'] = normalized_combinations  # Add new column to store normalized values


In [29]:
def is_child(outer, inner):
    outer_components = outer.split('+')
    inner_components = inner.split('+')
    
    return all(component in inner_components for component in outer_components)

# count group

In [30]:
def frequence_combination_integrating_group(df):
    # Create the result DataFrame
    result = pd.DataFrame(columns=["Nội dung hỗ trợ", "Reason Group", "Combination", "How many protocol in solution", "Success Count", "Consideration Count", "All Count"])
    
    # Iterate through the DataFrame
    for index, outer_row in df.iterrows():
        case_id = outer_row["case_id"]
        reason_group = outer_row["Reason Group"]
        combination = outer_row["Combination"]
        noidunghotro = outer_row["Nội dung hỗ trợ"]
        hmpis = outer_row["How many protocol in solution"]

        success_count = 0
        consideration_count = 0
        all_count = 0
        
        existing_row = result[
            (result["Reason Group"] == reason_group) & 
            (result["Combination"] == combination)
        ]

        if existing_row.empty:
            for inner_index, inner_row in df.iterrows():
                if (
                    combination == inner_row["Combination"]
                    and is_child(reason_group, inner_row["Reason Group"]) == True
                    and noidunghotro == inner_row["Nội dung hỗ trợ"]
                ):
                    all_count += 1
                    if inner_row['Solution Result'] == "Thành công":
                        success_count += 1
                    elif inner_row['Solution Result'] == "Cân nhắc":
                        consideration_count += 1
            
            # Append a new row to the result DataFrame
            result = pd.concat(
                [
                    result, 
                    pd.DataFrame(
                        {
                            "STT": [case_id],
                            "Nội dung hỗ trợ": [noidunghotro],
                            "Reason Group": [reason_group],
                            "Combination": [combination],
                            "How many protocol in solution": [hmpis],
                            "Success Count": [success_count],
                            "Consideration Count": [consideration_count],
                            "All Count": [all_count]
                        }
                    )
                ],
                ignore_index=True
            )

            # Remove the processed row from the original DataFrame
            df = df[~((df["Reason Group"] == reason_group) & (df["Combination"] == combination) & (df["Nội dung hỗ trợ"] == noidunghotro))]
            
    return result

In [31]:
reason_group = frequence_combination_integrating_group(df)

In [32]:
reason_group

,Nội dung hỗ trợ,Reason Group,Combination,How many protocol in solution,Success Count,Consideration Count,All Count,STT
0,Thanh lý hợp đồng internet,G1,ds1,1,0,1,9,1.0
1,Thanh lý hợp đồng internet,G1,ds3,1,0,3,7,1.0
2,Thanh lý hợp đồng internet,G1,dx5,1,2,1,28,1.0
3,Thanh lý hợp đồng internet,G3,hd5,1,1,0,6,1.0
4,Thanh lý hợp đồng internet,G0,ds1+dx4,2,0,1,4,2.0
...,...,...,...,...,...,...,...,...
976,Thanh lý hợp đồng truyền hình trong combo,G0,ds1+ll5,2,1,0,1,683.0
977,Thanh lý hợp đồng truyền hình trong combo,G2,ds2+ll2,2,1,0,1,683.0
978,Thanh lý hợp đồng truyền hình trong combo,G2,gt1+ll1+ll16+ll2,4,0,0,1,684.0
979,Thanh lý hợp đồng truyền hình trong combo,G5,ll3+ll5,2,0,0,1,684.0


# (count reason) Count the frequency of each combination, integrating unique 'n' values.

In [33]:
def frequence_combination_integrating_reason(df):

    result = pd.DataFrame(columns=["Nội dung hỗ trợ", "Reason Group", "Reason", "Combination", "How many protocol in solution", "Success Count", "Consideration Count"])
    
    for index, outer_row in df.iterrows():
        case_id = outer_row["case_id"]
        reason = outer_row["Reason"]
        reason_group = outer_row["Reason Group"]
        combination = outer_row["Combination"]
        noidunghotro = outer_row["Nội dung hỗ trợ"]
        hmpis = outer_row["How many protocol in solution"]

        success_count = 0
        consideration_count = 0
        all_count = 0
        
        existing_row = result[
            (result["Reason"] == reason) & 
            (result["Combination"] == combination)
        ]

        if existing_row.empty:
            for inner_index, inner_row in df.iterrows():
                if (
                    combination == inner_row["Combination"]
                    and is_child(reason, inner_row["Reason"]) == True
                    and noidunghotro == inner_row["Nội dung hỗ trợ"]
                ):
                    all_count += 1
                    if inner_row['Solution Result'] == "Thành công":
                        success_count += 1
                    elif inner_row['Solution Result'] == "Cân nhắc":
                        consideration_count += 1
            
            # Append a new row to the result DataFrame
            result = pd.concat(
                [
                    result, 
                    pd.DataFrame(
                        {
                            "STT": [case_id],
                            "Nội dung hỗ trợ": [noidunghotro],
                            "Reason Group": [reason_group],
                            "Reason": [reason],
                            "Combination": [combination],
                            "How many protocol in solution": [hmpis],
                            "Success Count": [success_count],
                            "Consideration Count": [consideration_count],
                            "All Count": [all_count]
                        }
                    )
                ],
                ignore_index=True
            )

            # Remove the processed row from the original DataFrame
            df = df[~((df["Reason"] == reason) & (df["Combination"] == combination) & (df["Nội dung hỗ trợ"] == noidunghotro))]
    
    return result

In [34]:
reason = frequence_combination_integrating_reason(df)

In [35]:
reason

,Nội dung hỗ trợ,Reason Group,Reason,Combination,How many protocol in solution,Success Count,Consideration Count,STT,All Count
0,Thanh lý hợp đồng internet,G1,n12,ds1,1,0,0,1.0,1.0
1,Thanh lý hợp đồng internet,G1,n12,ds3,1,0,0,1.0,1.0
2,Thanh lý hợp đồng internet,G1,n12,dx5,1,0,0,1.0,8.0
3,Thanh lý hợp đồng internet,G3,n34,hd5,1,0,0,1.0,2.0
4,Thanh lý hợp đồng internet,G0,n00,ds1+dx4,2,0,1,2.0,4.0
...,...,...,...,...,...,...,...,...,...
1256,Thanh lý hợp đồng truyền hình trong combo,G2,n21,ds2+ll2,2,1,0,683.0,1.0
1257,Thanh lý hợp đồng truyền hình trong combo,G2,n21,gt1+ll1+ll16+ll2,4,0,0,684.0,1.0
1258,Thanh lý hợp đồng truyền hình trong combo,G5,n50,ds1,1,0,0,684.0,1.0
1259,Thanh lý hợp đồng truyền hình trong combo,G5,n50,ll3+ll5,2,0,0,684.0,1.0


In [36]:
group_issue_des = dict(zip(nn['Mã nhóm nguyên nhân'], nn['Nhóm nguyên nhân']))
reason_group['Des Reason Group'] = reason_group['Reason Group'].map(group_issue_des)

group_issue_des = dict(zip(nn['Mã nhóm nguyên nhân'], nn['Nhóm nguyên nhân']))
reason['Des Reason Group'] = reason['Reason Group'].map(group_issue_des)
issue_des = dict(zip(nn['Tên nguyên nhân'], nn['Nguyên nhân']))
reason['Des Reason'] = reason['Reason'].map(issue_des)

In [38]:
output_file = 'store\\27Nov\count_combination_27Nov_v4.xlsx'

with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    reason.to_excel(writer, sheet_name='reason', index=False)
    reason_group.to_excel(writer, sheet_name='reason group', index=False)

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\anhthy\AppData\Local\Temp\ipykernel_2184\3876448333.py:1: SyntaxWarning: invalid escape sequence '\c'
  output_file = 'store\\27Nov\count_combination_27Nov_v4.xlsx'
